# Capstone Project - the possible covid-19 outbreak districts in Seattle

## 1. Introduction

The covid-19 vrius has become a major problem in the world, it is spreading at a fast speed, and can be transmitted thought human to human contact, hence, it is important to find district with lots of places that require human to human contact, such as restaurants. The study is done at a place that I used to live: seattle.

## 2. Data


The analysis to find the density of different venues in different district in Seattle:

The sources of data are the following to achieve their respective aims:

* I get the information about the district in seattle through the websitle https://seattle.findwell.com/seattle-neighborhoods/

* I look up the latitudes and longitudes of the districts through the OpenCage Geocoder API

* I get the location information thought the  Foursquare API.


for the sake of simplicity, I copied the information on the website and then pasted it to my excel file, and saved it as the csv format.

In [3]:
import pandas as pd
#!pip install lxml

#!pip install opencage
#from opencage.geocoder import OpenCageGeocode

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

#!conda install -c conda-forge folium --yes
import folium

import requests

import numpy as np

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

from pandas.io.json import json_normalize

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    folium-0.10.1              |             py_0          59 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTAL

I used `pandas` to read the files into dataframe

In [11]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_ad77cbcb57354cd191af72cc8edf6604 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='*************',
    ibm_auth_endpoint="*************",
    config=Config(signature_version='*************'),
    endpoint_url='*************')

body = client_*************.get_object(Bucket='c*************b',Key='seattle.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_seattle = pd.read_csv(body)
df_seattle.head()


,district,neighborhood
0,Belltown,Central Seattle
1,Broadmoor,Central Seattle
2,Capitol Hill,Central Seattle
3,Central District,Central Seattle
4,Denny-Blaine,Central Seattle


### Generate coordinate for All Districts

 OpenCage Geocoder,can be used to generate the latitude and the longitude coordinates of each districts needed for foursquare location data.

In [12]:

from opencage.geocoder import OpenCageGeocode
key="*************"
geocoder = OpenCageGeocode(key)


In [14]:
list_lat = []
list_long = []

for index, row in df_seattle.iterrows():
    
    district = row['district']
    neighborhood = row['neighborhood']       
    query = str(district)+','+str(neighborhood)
    
    geo_results = geocoder.geocode(query)   
    district_lat = geo_results[0]['geometry']['lat']
    district_long = geo_results[0]['geometry']['lng']
    
    list_lat.append(district_lat)
    list_long.append(district_long)
    
    
df_seattle['Latitude'] = list_lat
df_seattle['Longitude'] = list_long

df_seattle

,district,neighborhood,Latitude,Longitude
0,Belltown,Central Seattle,47.613231,-122.345361
1,Broadmoor,Central Seattle,47.635170,-122.289711
2,Capitol Hill,Central Seattle,47.619004,-122.320215
3,Central District,Central Seattle,47.603110,-122.308270
4,Denny-Blaine,Central Seattle,47.622327,-122.285985
5,Downtown,Central Seattle,47.604949,-122.333274
6,Eastlake,Central Seattle,47.643145,-122.326172
7,First Hill,Central Seattle,47.609305,-122.324984
8,International District,Central Seattle,47.602896,-122.339841
9,Leschi,Central Seattle,47.600535,-122.291866


In [16]:
print('The dataframe has {} regions and {} districts.'.format(
        len(df_seattle['neighborhood'].unique()),
        df_seattle.shape[0]
    )
)

The dataframe has 5 regions and 55 districts.


## 3. Methodology
By converting addresses into their equivalent latitude and longitude values. It is possible to use the Foursquare API to explore neighborhoods in Seattle. To explore function to get the most common venue categories in each neighborhood, and K-means clustering algorithm is used to group the neighborhoods into clusters.   Finally, the Folium library is used to visualize the neighborhoods in Seattle and their emerging cluster.

The area with the greatest density of huaman to human contact venues is the most vulnerable district. 


## 4. Analysis



#### Use geopy library to get the latitude and longitude values of SEATTLE



In [17]:
address = 'Seattle'

geolocator = Nominatim(user_agent="seatt")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Seattle are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Seattle are 47.6038321, -122.3300624.


In [50]:
# create map of Seattle using latitude and longitude values
map_Sea = folium.Map(location=[latitude, longitude],zoom_start=10)

# add markers to map
for lat, lng, region, district in zip(df_seattle['Latitude'], df_seattle['Longitude'], df_seattle['neighborhood'], df_seattle['district']):
    label = '{}, {}'.format(district, region)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Sea)  

map_Sea

#### Define Foursquare Credentials and Version

In [51]:
CLIENT_ID = 'J*************R' # your Foursquare ID
CLIENT_SECRET = 'G4**************************5' # your Foursquare Secret
VERSION = '20200412' # Foursquare API version
LIMIT = 100

In [52]:
# get the district's name
df_seattle.loc[0, 'district']

'Belltown'

In [53]:

district_latitude = df_seattle.loc[0, 'Latitude']
district_longitude = df_seattle.loc[0, 'Longitude']

district_name = df_seattle.loc[0, 'district']

print('Latitude and longitude values of {} are {}, {}.'.format(district_name, 
                                                               district_latitude, 
                                                               district_longitude))

Latitude and longitude values of Belltown are 47.6132311, -122.3453607.


#### Getting Top 100 Venues in the First District within Radius of 500 Meters


In [54]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    district_latitude, 
    district_longitude, 
    radius, 
    LIMIT)

url_results = requests.get(url).json()
url_results

{'meta': {'code': 200, 'requestId': '5e9369ad949393001b42b9da'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Belltown',
  'headerFullLocation': 'Belltown, Seattle',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 97,
  'suggestedBounds': {'ne': {'lat': 47.6177311045, 'lng': -122.33869790467669},
   'sw': {'lat': 47.6087310955, 'lng': -122.35202349532331}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5813f76238fa674d1ed9bb06',
       'name': 'FOB Poke Bar',
       'location': {'address': '220 Blanchard St',
        'lat': 47.613829,
        'lng': -122.343809,
        'labeledLatLngs': [{'label': 'display',
          '

In [55]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [56]:
venues = url_results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,FOB Poke Bar,Poke Place,47.613829,-122.343809
1,Cloudburst Brewing,Brewery,47.611565,-122.345212
2,Have A Heart Belltown,Marijuana Dispensary,47.612860,-122.344403
3,Rocco's,Pizza Place,47.613956,-122.345304
4,Umi Sake House,Sushi Restaurant,47.613285,-122.346111


In [57]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

97 venues were returned by Foursquare.


In [58]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        url_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in url_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['district', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [59]:
venues = getNearbyVenues(names=df_seattle['district'],
                                   latitudes=df_seattle['Latitude'],
                                   longitudes=df_seattle['Longitude']
                                  )

Belltown
Broadmoor
Capitol Hill
Central District
Denny-Blaine
Downtown
Eastlake
First Hill
International District
Leschi
Madison Park
Madison Valley
Madrona
Montlake
Pioneer Square
South Lake Union
Bryant
Hawthorne Hills
Lake City
Laurelhurst
Maple Leaf
Matthews Beach
Northgate
Ravenna
Roosevelt
U-District
View Ridge
Wedgwood
Windermere
Ballard
Blue Ridge/North Beach
Broadview
Crown Hill
Fremont
Green Lake
Greenwood
Haller Lake
Loyal Heights
Magnolia
Phinney Ridge
Queen Anne
Wallingford
Westlake
Whittier Heights
Beacon Hill
Columbia City
Georgetown
Mount Baker
Rainier Beach
Seward Park
South Park
Admiral
Alki
Fauntleroy
Junction


In [60]:
print(venues.shape)
venues.head()

(1569, 7)


,district,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Belltown,47.613231,-122.345361,FOB Poke Bar,47.613829,-122.343809,Poke Place
1,Belltown,47.613231,-122.345361,Cloudburst Brewing,47.611565,-122.345212,Brewery
2,Belltown,47.613231,-122.345361,Have A Heart Belltown,47.612860,-122.344403,Marijuana Dispensary
3,Belltown,47.613231,-122.345361,Rocco's,47.613956,-122.345304,Pizza Place
4,Belltown,47.613231,-122.345361,Umi Sake House,47.613285,-122.346111,Sushi Restaurant


Numbers of Venues Returned for Each District:

In [61]:
venues.groupby('district').count()

,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
district,,,,,,
Admiral,43,43,43,43,43,43
Alki,43,43,43,43,43,43
Ballard,8,8,8,8,8,8
Beacon Hill,16,16,16,16,16,16
Belltown,97,97,97,97,97,97
Blue Ridge/North Beach,41,41,41,41,41,41
Broadmoor,4,4,4,4,4,4
Broadview,9,9,9,9,9,9
Bryant,3,3,3,3,3,3


Examining Each District

In [62]:
# one hot encoding
district_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
district_onehot['district'] = venues['district'] 

# move neighborhood column to the first column
fixed_columns = [district_onehot.columns[-1]] + list(district_onehot.columns[:-1])
district_onehot = district_onehot[fixed_columns]

district_onehot.head()

,district,ATM,American Restaurant,Amphitheater,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Belltown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Belltown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Belltown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Belltown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Belltown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
district_onehot.shape

(1569, 255)

#### Group Rows by District and by Taking the Mean of the Frequency of Occurrence of Each Category

In [64]:
Sea_grouped = district_onehot.groupby('district').mean().reset_index()
Sea_grouped

,district,ATM,American Restaurant,Amphitheater,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Admiral,0.000000,0.023256,0.00,0.000000,0.000000,0.00,0.023256,0.000000,0.023256,...,0.000000,0.023256,0.023256,0.000000,0.023256,0.000000,0.000000,0.023256,0.000000,0.000000
1,Alki,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.046512,0.000000,0.000000,...,0.000000,0.000000,0.023256,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Ballard,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Beacon Hill,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000
4,Belltown,0.000000,0.020619,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.010309,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Blue Ridge/North Beach,0.000000,0.024390,0.00,0.000000,0.000000,0.00,0.000000,0.024390,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Broadmoor,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Broadview,0.000000,0.000000,0.00,0.111111,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Bryant,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000
9,Capitol Hill,0.010000,0.040000,0.00,0.000000,0.000000,0.01,0.000000,0.000000,0.010000,...,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000


In [65]:
district_grouped.shape

(55, 255)

#### Print Each District along with the Top 5 Most Common Venues

In [66]:
num_top_venues = 5

for hood in Sea_grouped['district']:
    print("----"+hood+"----")
    temp = district_grouped[Sea_grouped['district'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Admiral----
                       venue  freq
0                Coffee Shop  0.12
1              Grocery Store  0.05
2                        Pub  0.05
3                Gas Station  0.02
4  Middle Eastern Restaurant  0.02


----Alki----
                venue  freq
0      Ice Cream Shop  0.09
1         Coffee Shop  0.07
2  Mexican Restaurant  0.07
3  Italian Restaurant  0.05
4         Art Gallery  0.05


----Ballard----
               venue  freq
0      Jewelry Store  0.12
1                Bar  0.12
2        Coffee Shop  0.12
3  Food & Drink Shop  0.12
4  French Restaurant  0.12


----Beacon Hill----
                      venue  freq
0                      Café  0.12
1  Mediterranean Restaurant  0.06
2               Coffee Shop  0.06
3        Mexican Restaurant  0.06
4               Bus Station  0.06


----Belltown----
            venue  freq
0           Hotel  0.06
1     Coffee Shop  0.05
2  Breakfast Spot  0.05
3             Bar  0.05
4          Bakery  0.05


----Blue Ridge/North

In [67]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create a New Dataframe & Display the Top 10 Venues for Each District

In [68]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['district']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
district_venues_sorted = pd.DataFrame(columns=columns)
district_venues_sorted['district'] = Sea_grouped['district']

for ind in np.arange(Sea_grouped.shape[0]):
    district_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Sea_grouped.iloc[ind, :], num_top_venues)

district_venues_sorted

,district,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Admiral,Coffee Shop,Grocery Store,Pub,Italian Restaurant,Fast Food Restaurant,Salon / Barbershop,Cocktail Bar,Convenience Store,Pizza Place,Pharmacy
1,Alki,Ice Cream Shop,Mexican Restaurant,Coffee Shop,Italian Restaurant,Park,Art Gallery,Restaurant,Thai Restaurant,Fish & Chips Shop,Beach Bar
2,Ballard,Baseball Field,Coffee Shop,Jewelry Store,Park,Gift Shop,French Restaurant,Bar,Food & Drink Shop,Fish Market,Field
3,Beacon Hill,Café,Coffee Shop,Gas Station,Scenic Lookout,Taco Place,Peking Duck Restaurant,Mediterranean Restaurant,Mexican Restaurant,Bakery,Pizza Place
4,Belltown,Hotel,Bakery,Coffee Shop,Breakfast Spot,Bar,Sushi Restaurant,Seafood Restaurant,Italian Restaurant,New American Restaurant,Pizza Place
5,Blue Ridge/North Beach,Hotel,Coffee Shop,Sandwich Place,Concert Hall,Gym,Cocktail Bar,Café,Italian Restaurant,Social Club,Lounge
6,Broadmoor,Garden,Playground,Soccer Field,Golf Course,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flower Shop,Zoo Exhibit
7,Broadview,Convenience Store,Furniture / Home Store,Antique Shop,Thai Restaurant,Sushi Restaurant,Video Store,Pizza Place,Food Truck,Beer Bar,Food & Drink Shop
8,Bryant,Wine Shop,Organic Grocery,Park,Zoo Exhibit,Farmers Market,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand
9,Capitol Hill,Coffee Shop,Bar,American Restaurant,Mexican Restaurant,Japanese Restaurant,Thrift / Vintage Store,Pizza Place,Cocktail Bar,Café,Taco Place


###  Clustering  

#### use k*-means to cluster the districts 

In [69]:
# set number of clusters
kclusters = 6

clustering = Sea_grouped.drop('district', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 4, 2, 0, 2], dtype=int32)

#### Create a New Dataframe for the Cluster and the Top 10 Venues for Each District

In [73]:
# add clustering labels
district_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

district_merged = df_seattle

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
district_merged = district_merged.join(district_venues_sorted.set_index('district'), on='district', how='right')

district_merged.head() # check the last columns!

,district,neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Belltown,Central Seattle,47.613231,-122.345361,2,Hotel,Bakery,Coffee Shop,Breakfast Spot,Bar,Sushi Restaurant,Seafood Restaurant,Italian Restaurant,New American Restaurant,Pizza Place
1,Broadmoor,Central Seattle,47.635170,-122.289711,4,Garden,Playground,Soccer Field,Golf Course,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flower Shop,Zoo Exhibit
2,Capitol Hill,Central Seattle,47.619004,-122.320215,2,Coffee Shop,Bar,American Restaurant,Mexican Restaurant,Japanese Restaurant,Thrift / Vintage Store,Pizza Place,Cocktail Bar,Café,Taco Place
3,Central District,Central Seattle,47.603110,-122.308270,2,Coffee Shop,Performing Arts Venue,Breakfast Spot,Plaza,Playground,Vietnamese Restaurant,Art Gallery,Pizza Place,Chinese Restaurant,BBQ Joint
4,Denny-Blaine,Central Seattle,47.622327,-122.285985,1,Park,Surf Spot,Monument / Landmark,Beach,Food & Drink Shop,Field,Fish & Chips Shop,Fish Market,Flower Shop,Zoo Exhibit


#### Visualize the Resulting Clusters

In [74]:
# create map
map_clusters = folium.Map(location=[latitude+0.08, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(district_merged['Latitude'], district_merged['Longitude'], district_merged['district'], district_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

###  Examining Clusters

Each cluster is examined and the discriminating venue categories that distinguish each cluster are determined. 

#### Cluster 1 -low probility district

In [75]:
district_merged.loc[district_merged['Cluster Labels'] == 0, district_merged.columns[[1] + [2] + list(range(5, district_merged.shape[1]))]]

,neighborhood,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Central Seattle,47.600535,Park,Playground,Pizza Place,Grocery Store,Pet Store,Bus Stop,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market
16,Northeast Seattle,47.675233,Wine Shop,Organic Grocery,Park,Zoo Exhibit,Farmers Market,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand
19,Northeast Seattle,47.663432,Pharmacy,Café,Music Venue,Lawyer,Bus Stop,Park,Food & Drink Shop,Fish & Chips Shop,Fish Market,Flower Shop
21,Northeast Seattle,47.696927,Park,Vacation Rental,Trail,Playground,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Zoo Exhibit
36,Northwest Seattle,47.719748,Playground,Park,Dance Studio,Coffee Shop,Lake,Zoo Exhibit,Field,Fish & Chips Shop,Fish Market,Flower Shop
45,South Seattle,47.559747,Park,Construction & Landscaping,Mediterranean Restaurant,Gym,Light Rail Station,Tailor Shop,Pet Store,Farmers Market,Thai Restaurant,Soccer Field
48,South Seattle,47.522421,Light Rail Station,Deli / Bodega,Plaza,Tennis Court,Mexican Restaurant,Playground,Food Stand,Food & Drink Shop,Food Truck,Farmers Market


#### Cluster 2 - low pobility district 

In [76]:
district_merged.loc[district_merged['Cluster Labels'] == 1, district_merged.columns[[1] + [2] + list(range(5, district_merged.shape[1]))]]

,neighborhood,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Seattle,47.622327,Park,Surf Spot,Monument / Landmark,Beach,Food & Drink Shop,Field,Fish & Chips Shop,Fish Market,Flower Shop,Zoo Exhibit
26,Northeast Seattle,47.679543,Park,Construction & Landscaping,Zoo Exhibit,Farmers Market,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand
34,Northwest Seattle,47.678284,Park,Theater,Trail,Beach,Lake,Tennis Court,Fish & Chips Shop,Fast Food Restaurant,Field,Zoo Exhibit
49,South Seattle,47.554914,Park,Amphitheater,Trail,Zoo Exhibit,Food & Drink Shop,Field,Fish & Chips Shop,Fish Market,Flower Shop,Food Stand


## Cluster 3 - high probility districts.

In [77]:
district_merged.loc[district_merged['Cluster Labels'] == 2, district_merged.columns[[1] + [2] + list(range(5, district_merged.shape[1]))]]

,neighborhood,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Seattle,47.613231,Hotel,Bakery,Coffee Shop,Breakfast Spot,Bar,Sushi Restaurant,Seafood Restaurant,Italian Restaurant,New American Restaurant,Pizza Place
2,Central Seattle,47.619004,Coffee Shop,Bar,American Restaurant,Mexican Restaurant,Japanese Restaurant,Thrift / Vintage Store,Pizza Place,Cocktail Bar,Café,Taco Place
3,Central Seattle,47.603110,Coffee Shop,Performing Arts Venue,Breakfast Spot,Plaza,Playground,Vietnamese Restaurant,Art Gallery,Pizza Place,Chinese Restaurant,BBQ Joint
5,Central Seattle,47.604949,Coffee Shop,Hotel,Cocktail Bar,Sandwich Place,Seafood Restaurant,Italian Restaurant,Breakfast Spot,Park,Pier,Pizza Place
6,Central Seattle,47.643145,Sandwich Place,Coffee Shop,Mexican Restaurant,Playground,Italian Restaurant,Steakhouse,Bar,Burger Joint,Deli / Bodega,Furniture / Home Store
7,Central Seattle,47.609305,Sandwich Place,Coffee Shop,Hotel,Pharmacy,Pizza Place,Bakery,Asian Restaurant,Restaurant,Poke Place,Indian Restaurant
8,Central Seattle,47.602896,Seafood Restaurant,Coffee Shop,Gift Shop,Burger Joint,Park,Boat or Ferry,Clothing Store,Cocktail Bar,Pier,Theme Park Ride / Attraction
10,Central Seattle,47.635930,Bar,Park,Bank,Kitchen Supply Store,Café,Mexican Restaurant,Food & Drink Shop,Coffee Shop,Beach,Salon / Barbershop
11,Central Seattle,47.619916,Italian Restaurant,French Restaurant,Coffee Shop,Sushi Restaurant,Bar,Pet Store,Kids Store,Dessert Shop,Vegetarian / Vegan Restaurant,Gym
12,Central Seattle,47.612792,Asian Restaurant,Food Truck,Bus Stop,Farmers Market,Coffee Shop,Italian Restaurant,Gift Shop,Seafood Restaurant,Grocery Store,Tapas Restaurant


#### Cluster 4 - moderate probility district

In [78]:
district_merged.loc[district_merged['Cluster Labels'] == 3, district_merged.columns[[1] + [2] + list(range(5, district_merged.shape[1]))]]

,neighborhood,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Northeast Seattle,47.672043,American Restaurant,Bank,Hardware Store,Massage Studio,Café,Fried Chicken Joint,French Restaurant,Food Truck,Frozen Yogurt Shop,Food Stand
50,South Seattle,47.561488,Farmers Market,American Restaurant,Café,Park,Fast Food Restaurant,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck


#### Cluster 5 - low probility district

In [79]:
district_merged.loc[district_merged['Cluster Labels'] == 4, district_merged.columns[[1] + [2] + list(range(5, district_merged.shape[1]))]]

,neighborhood,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Central Seattle,47.63517,Garden,Playground,Soccer Field,Golf Course,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flower Shop,Zoo Exhibit


#### Cluster 6 - low probility district


In [80]:
district_merged.loc[district_merged['Cluster Labels'] == 5, district_merged.columns[[1] + [2] + list(range(5, district_merged.shape[1]))]]

,neighborhood,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Northeast Seattle,47.670932,Park,Gym,Pizza Place,Greek Restaurant,Zoo Exhibit,Fish Market,Field,Fish & Chips Shop,Flower Shop,Farmers Market


## 5. Results

The resulting clusters are as follow :

* Cluster 1 - low probility district
* Cluster 2 - low probility district
* Cluster 3 - High probility district
* Cluster 4 - moderate probility district
* Cluster 5 - low probility district
* Cluster 6 - low probility district

From the result, Cluster 3 has the most density of cafe shop and restrants **Therefore, Cluster 3 has the highest potential to have the covid-19 outbreak.**
